# Freeform Programming Assessment - Data Trimming

The dataset I used was huge - over 700MB - so could not be pushed to GitHub or uploaded to Turnitin.

This notebook shows the code I used to trim the data down.

This notebook uses data from the OpenPowerlifting project, https://www.openpowerlifting.org.

You may download a copy of the data at https://data.openpowerlifting.org or https://openpowerlifting.gitlab.io/opl-csv/bulk-csv.html.

OpenPowerlifting is a community service project to create a permanent, open archive of the world's powerlifting data. All data available on the website is contributed to the public domain.

In [1]:
import pandas as pd

powerlifting_data = pd.read_csv("openpowerlifting-2025-11-22-823f23d6.csv", low_memory=False)

An immediate way to trim down ~30% of the file size, while still allowing a good analysis, was to restrict the dataset to male lifters.

Analysing only male lifters will also add fewer confounding variables to the analysis.

Data was also restricted to be between 2005 and 2024. This not only saves more file size, but is beneficial as older records are less accurate, and more incomplete. 

In general, modern sporting events are also drug tested more rigorously - the 2005-2024 timeframe will have more 'natty' lifters.

Limiting analysis to 2005-2024 improves data quality, and represents modern, standardised competitive powerlifting. 

In [2]:
# Selecting male powerlifters
male = powerlifting_data[powerlifting_data["Sex"] == "M"].copy()

# Convert Date to datetime and create Year column
male["Date"] = pd.to_datetime(male["Date"], errors="coerce")
male["Year"] = male["Date"].dt.year

# Filter by year - we want 2005 to 2024
male_recent = male[(male["Year"] >= 2005) & (male["Year"] <= 2024)].copy()

print(male_recent.shape)


(2150225, 43)


In [3]:
# 4 metric columns that are not needed, and the sex column since we now only have males
# can also remove individual lift attempts, as only the best lifts and the SBD total are needed
male_smaller = male_recent.drop(columns=[
    "Dots", "Wilks", "Glossbrenner", "Goodlift", "Sex", "Squat1Kg", "Squat2Kg","Squat3Kg", "Squat4Kg", "Bench1Kg", "Bench2Kg", "Bench3Kg", "Bench4Kg", "Deadlift1Kg", "Deadlift2Kg", "Deadlift3Kg", "Deadlift4Kg"
])

In [4]:
male_smaller.columns
# still have plenty of data to work with

Index(['Name', 'Event', 'Equipment', 'Age', 'AgeClass', 'BirthYearClass',
       'Division', 'BodyweightKg', 'WeightClassKg', 'Best3SquatKg',
       'Best3BenchKg', 'Best3DeadliftKg', 'TotalKg', 'Place', 'Tested',
       'Country', 'State', 'Federation', 'ParentFederation', 'Date',
       'MeetCountry', 'MeetState', 'MeetTown', 'MeetName', 'Sanctioned',
       'Year'],
      dtype='object')

In [5]:
# can also get rid of rows where event isn't SBD
male_sbd = male_smaller[male_smaller["Event"] == "SBD"].copy()

male_sbd.head()

,Name,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,Best3SquatKg,...,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName,Sanctioned,Year
332,Nauris Kalašņikovs,SBD,Wraps,NaN,13-15,NaN,T1,38.8,44,50.0,...,NaN,WPC-Latvia,WPC,2011-12-17,Latvia,NaN,Dobele,Latvian Championships,Yes,2011
333,Raivo Berkins,SBD,Wraps,NaN,NaN,NaN,Y,42.8,44,35.0,...,NaN,WPC-Latvia,WPC,2011-12-17,Latvia,NaN,Dobele,Latvian Championships,Yes,2011
334,Dāvis Vilks,SBD,Wraps,NaN,13-15,NaN,T1,47.0,48,50.0,...,NaN,WPC-Latvia,WPC,2011-12-17,Latvia,NaN,Dobele,Latvian Championships,Yes,2011
335,Gvido Granauskis,SBD,Wraps,NaN,13-15,NaN,T1,51.7,52,55.0,...,NaN,WPC-Latvia,WPC,2011-12-17,Latvia,NaN,Dobele,Latvian Championships,Yes,2011
336,Dinārs Skuja,SBD,Wraps,NaN,13-15,NaN,T1,52.8,56,55.0,...,NaN,WPC-Latvia,WPC,2011-12-17,Latvia,NaN,Dobele,Latvian Championships,Yes,2011


Initially I had to remove over half the columns in order to reduce the file size.

However, the best workaround I could find was to save the data as a compressed csv file.

In [7]:
# saving the new dataframe to a csv file
male_sbd.to_csv("male_powerlifting_2005_2024.csv.gz", index=False, compression="gzip")